# Operacje na ramkach danych

In [77]:
import pandas as pd
from pandas import DataFrame, Series
from numpy import nan

## Usuwanie brakujących obserwacji

Czasami w naszym zbiorze danych znajdują się brakujące wartości. W takim wypadku chcielibyśmy je zidentyfikować oraz usunąć. W pakiecie `pandas` brakujące wartości oznaczane są obiektem `nan`, co jest skrótem od "not a number". W R odpowiednikiem jest wartość `NA`. Przy wczytywaniu danych warto zadbać o to, żeby prawidłowo wczytać również te obserwacje, które są niekompletne (więcej o tym w notebooku  poświęconym wczytywaniu danych). Na potrzeby ćwiczenia stworzymy ramkę danych z brakującymi wartościami.

In [78]:
data = DataFrame({
    'uczestnik': [1,2,3,4,5,6,7,8,9,10],
    'wiek' : [26,34,nan,25,56,nan,23,21,19,10],
    'płeć' : ['K','M','K','M',nan,'K','M','K', nan, 'M'],
})
data

,uczestnik,wiek,płeć
0,1,26.0,K
1,2,34.0,M
2,3,NaN,K
3,4,25.0,M
4,5,56.0,NaN
5,6,NaN,K
6,7,23.0,M
7,8,21.0,K
8,9,19.0,NaN
9,10,10.0,M


Pierwszym naszym zadaniem jest zidentyfikowanie niekompletnych obserwacji. Możemy posłużyć się metodą `isna` wywołaną na ramce danych.

In [79]:
data.isna()

,uczestnik,wiek,płeć
0,False,False,False
1,False,False,False
2,False,True,False
3,False,False,False
4,False,False,True
5,False,True,False
6,False,False,False
7,False,False,False
8,False,False,True
9,False,False,False


Aby zobaczyć tylko niekompletne obserwacje możemy wykorzystać naszą wiedzę dotyczącą filtrowania danych. Za pomocą metody `any` (z argumentem `1`, który mówi, że szukamy przynajmniej jednej takiej wartości wierszami) możemy sprawdzić, w których wierszach znajduje się przynajmniej jedna brakująca obserwacja:

In [80]:
data.isna().any(1)

0    False
1    False
2     True
3    False
4     True
5     True
6    False
7    False
8     True
9    False
dtype: bool

Teraz możemy wyświetlić tylko niekompletne obserwacje i np. przyjrzeć się im bliżej:

In [81]:
data[data.isna().any(1)]

,uczestnik,wiek,płeć
2,3,NaN,K
4,5,56.0,NaN
5,6,NaN,K
8,9,19.0,NaN


Jeżeli chcemy po prostu usunąć wszystkie wiersze, w których mamy przynajmniej jedną brakujacą wartość, wystarczy wywołać na ramce danych metodę `dropna`.

In [82]:
data.dropna()

,uczestnik,wiek,płeć
0,1,26.0,K
1,2,34.0,M
3,4,25.0,M
6,7,23.0,M
7,8,21.0,K
9,10,10.0,M


Czasami może się jednak zdarzyć, że za niekompletne obserwacje nie chcemy uważać wszystkich wierszy, w których brakuje jakiejkolwiek wartości. Powiedzmy, że interesują nas takie wiersze, w których brakuje co najwyżej jednej wartości. Możemy przekazać w metodzie `dropna` argument `thresh`, dzięki któremu ustalimy odpowiedni próg. W poniższym przykładzie `thresh=2` oznacza, że chcemy uzyskać tylko te obserwacje, gdzie w przynajmniej 2 kolumnach nie mamy brakujących wartości.

In [83]:
data.dropna(thresh=2)

,uczestnik,wiek,płeć
0,1,26.0,K
1,2,34.0,M
2,3,NaN,K
3,4,25.0,M
4,5,56.0,NaN
5,6,NaN,K
6,7,23.0,M
7,8,21.0,K
8,9,19.0,NaN
9,10,10.0,M


Może się również zdarzyć tak, że niektóre kolumny mniej nas interesują niż inne. Załóżmy, ze w naszym zbiorze mamy także kolumnę, w której są nieobowiązkowe komentarze badanych:

In [84]:
data['komentarz '] = ['Fajne badanie!', 
                    'Brak', 
                    'HELLO', 
                    nan, 
                    'Trochę długa była ta ankieta', 
                    nan,
                    nan,
                    'Nie mam',
                    nan,
                    'pzdr']
data

,uczestnik,wiek,płeć,komentarz
0,1,26.0,K,Fajne badanie!
1,2,34.0,M,Brak
2,3,NaN,K,HELLO
3,4,25.0,M,NaN
4,5,56.0,NaN,Trochę długa była ta ankieta
5,6,NaN,K,NaN
6,7,23.0,M,NaN
7,8,21.0,K,Nie mam
8,9,19.0,NaN,NaN
9,10,10.0,M,pzdr


Przy odsiewaniu niekompletnych obserwacji chcielibyśmy zignorować kolumnę `comment`. W tym celu możemy po prostu wskazać, które kolumny mają być brane pod uwagę przy decydowaniu, które obserwacje uznane mają być za niekompletne. Aby to zrobić, musimy jako argument `subset` przekazać listę z nazwami relewantnych kolumn:

In [85]:
data.dropna(subset = ['płeć', 'wiek'])

,uczestnik,wiek,płeć,komentarz
0,1,26.0,K,Fajne badanie!
1,2,34.0,M,Brak
3,4,25.0,M,NaN
6,7,23.0,M,NaN
7,8,21.0,K,Nie mam
9,10,10.0,M,pzdr


## Zmiana nazw kolumn

Załóżmy, że nasze dane chcielibyśmy mieć zapisane w dwóch językach - po polsku i po angielsku. Aby to osiągnąć musimy zmienić nazwy kolumn. W tym celu moglibyśmy po prostu nadpisać je ustawiając wartość zmiennej `data.columns`. Jeśli chcemy to zrobić selektywnie, lepiej posłużyć się metodą `rename` obiektów klasy `DataFrame`. W argumencie `columns` możemy przekazać słownik, w którym klucze są starymi nazwami a wartości nowymi:

In [86]:
data.rename(columns= {'uczestnik' : 'participant',
                     'wiek' : 'age',
                     'płeć' : 'sex',
                     'komentarz' : 'comment'},
           inplace=True) # domyślnie metody te zwracają nową ramkę, inplace pozwala zmodyfikować już istniejącą
data

,participant,age,sex,komentarz
0,1,26.0,K,Fajne badanie!
1,2,34.0,M,Brak
2,3,NaN,K,HELLO
3,4,25.0,M,NaN
4,5,56.0,NaN,Trochę długa była ta ankieta
5,6,NaN,K,NaN
6,7,23.0,M,NaN
7,8,21.0,K,Nie mam
8,9,19.0,NaN,NaN
9,10,10.0,M,pzdr


## Przekodowywanie wartości w ramce danych

Metoda `map` obiektów klasy `Series` pozwala na bardzo intuicyjne "przekodowywanie" wartości w interesujących nas kolumnach. Możemy na przykłąd zmienić wszystkie wartośći `K` na `Female` oraz `M` na `Male`. Sposób działania tej funkcji jest bardzo podobny, co w przypadku `rename` - możemy przekazać odpowiedni słownik.

In [88]:
data['sex'].map({'K' : 'Female', 'M' : 'Male'})

0    Female
1      Male
2    Female
3      Male
4       NaN
5    Female
6      Male
7    Female
8       NaN
9      Male
Name: sex, dtype: object

Jeżeli interesują nas bardziej zaawansowane operacje, możemy zamiast słownika przekazać tej metodzie funkcje. Wtedy funkcja ta będzie aplikowana do każdego elementu `Series`. W poniższym przykładzie chcielibyśmy wszystkie wartości z kolumny `sex` zmienić na małe litery i zastąpić je wartością "brak danych" tam, gdzie występuje wartość `nan`:

In [87]:
data['sex'].map(lambda x: x.lower() if type(x) == str else 'brak danych')

0              k
1              m
2              k
3              m
4    brak danych
5              k
6              m
7              k
8    brak danych
9              m
Name: sex, dtype: object

## Łączenie ramek danych

`pandas` oferuje bardzo rozbudowane możliwości złączania ramek danych (zewnętrzne, wewnętrzne, lewostronne i prawostronne otwarte). Zainteresowanych odsyłam do dokumentacji (wykracza to BARDZO poza tematykę kursu):

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html

My zajmiemy się najprostszym łączeniem ramek danych za pomocą funkcji `concat`. Załóżmy, że dla każdego badanego mamy osobny plik z danymi (a więc i osobną ramkę danych). Jest to sytuacja bardzo częsta (np. PsychoPy w taki sposób zapisuje dane).

In [101]:
P1 = DataFrame({
    'participant' : [1] * 3,
    'trialN' : range(1,4),
    'rt' : [345, 321, 287]
})

P2 = DataFrame({
    'participant' : [2] * 3,
    'trialN' : range(1,4),
    'rt' : [213, 432, 553]
})

In [102]:
P1

,participant,trialN,rt
0,1,1,345
1,1,2,321
2,1,3,287


In [103]:
P2

,participant,trialN,rt
0,2,1,213
1,2,2,432
2,2,3,553


Najprostszym sposobem połączenia tych ramek danych jest użycie funkcji `concat`. Przyjmuje ona listę, której elementami są ramki danych.

In [105]:
data = pd.concat([P1, P2], 
          axis=0) # oś w której chcemy połączyć ramki, `1` połączy je w szerz
data

,participant,trialN,rt
0,1,1,345
1,1,2,321
2,1,3,287
0,2,1,213
1,2,2,432
2,2,3,553


Po wykonaniu takiej operacji dobrze jest zresetować index (proszę zwrócić uwagę, że teraz indeksy nie są unikatowe).

In [108]:
data.reset_index(drop=True) # drop=True sprawia, że stary indeks nie staje się dodatkową kolumną

,participant,trialN,rt
0,1,1,345
1,1,2,321
2,1,3,287
3,2,1,213
4,2,2,432
5,2,3,553
